# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("weatherData.csv")
df.head()

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,hermanus,-34.42,19.23,61.00,52,1017,1.01,13
1,belmonte,-15.86,-38.88,69.80,94,1016,6.93,20
2,chokurdakh,70.63,147.92,18.97,96,1003,4.63,13
3,provideniya,64.38,-173.30,24.80,73,1010,4.47,90
4,longyearbyen,78.22,15.64,32.00,95,1010,21.92,90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 # Store latitude and longitude in locations
locations = df[["latitude", "longitude"]]

# Fill NaN values and convert to float
humid = df["humidity"]

In [4]:
 # Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=True, point_radius=8)


# Add layer
fig.add_layer(heat_layer)

# Format
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid red',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
heat_layer.max_intensity = 100
heat_layer.point_radius = 10
heat_layer.gradient = ['silver', 'blue', 'red']
heat_layer.opacity = 0.3
gmaps.figure(layout=figure_layout)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Retrieving Perfect Weather
mask1 = (df.temperature >= 70) & (df.temperature < 80)
mask2 = df.wind_speed < 10
mask3 = df.cloudiness == 0
mask4 = mask1 & mask2 & mask3

ideal_cities = df.loc[mask4].reset_index(drop=True)
ideal_cities

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,baghdad,33.34,44.40,75.20,47,1010,4.70,0
1,katsuura,35.13,140.30,71.78,51,1019,5.99,0
2,esna,25.29,32.55,71.60,43,1012,4.70,0
3,khawhai,23.37,93.12,74.70,40,1012,3.96,0
4,jinchang,38.50,102.17,79.99,7,1010,6.35,0
5,umm lajj,25.02,37.27,77.25,51,1010,1.59,0
6,benguela,-12.58,13.41,76.82,71,1015,3.27,0
7,namibe,-15.20,12.15,75.94,62,1014,5.61,0


In [6]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

coordinates = ideal_cities[["latitude", "longitude"]]

# Format
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid red',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
heat_layer.max_intensity = 100
heat_layer.point_radius = 5
heat_layer.opacity = 0.3
gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid red', height='600px', margin='0 auto 0 auto', padding='1px', widt…

In [7]:
# Retrieving the city name
city_info = []

for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Name</dt><dd>{row.cities}</dd>
                <dt>Temp</dt><dd>{row.temperature}</dd>
                <dt>Cloudiness</dt><dd>{row.cloudiness}%</dd>
                </dl>
                """
    
    city_info.append(info_box)

In [8]:
# Format
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid red',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
heat_layer.max_intensity = 100
heat_layer.point_radius = 5
heat_layer.gradient = ['silver', 'blue', 'red']
heat_layer.opacity = 0.3
gmaps.figure(layout=figure_layout)

# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)
coordinates = ideal_cities[["latitude", "longitude"]]

marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid red', height='600px', margin='0 auto 0 auto', padding='1px', widt…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
def getCountryForCity(lat, long):    
    rtnCountry = ""
    
    params = {"key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    lat = ideal_cities["latitude"][indx]
    lng = ideal_cities["longitude"][indx]

    # update address key value
    params['latlng'] = f"{lat},{lng}"

    # make request
    thing = requests.get(base_url, params=params)

    thing = thing.json()
    thing["results"][0]["address_components"]


    for addressComp in thing["results"][0]["address_components"]:
        if addressComp["types"][0] == "country":
            rtnCountry = addressComp["long_name"]
            
    return rtnCountry

In [10]:
countries = []
for indx, row in ideal_cities.iterrows():
    countries.append(getCountryForCity(row.latitude, row.longitude))

In [11]:
# Use the lat/lng we recovered to identify airports

names = []
addresses = []
ratings = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in ideal_cities.iterrows():
    # Variables (lat & Lng)
    lat = row["latitude"]
    lng = row["longitude"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Request
    name_address = requests.get(base_url, params=params)

    # Convert to json
    name_address = name_address.json()
    
    # Append and skip error
    try:
        names.append(name_address["results"][0]["name"])
        addresses.append(name_address["results"][0]["vicinity"])
        ratings.append(name_address["results"][0]["rating"])
    except (KeyError, IndexError):
        print("Not Found")
        names.append("")
        addresses.append("")
        ratings.append("")

Not Found


In [12]:
# New DataFrame
ideal_cities["country"] = countries
ideal_cities["hotels"] = names
ideal_cities["addresses"] = addresses
ideal_cities["ratings"] = ratings

ideal_cities

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness,country,hotels,addresses,ratings
0,baghdad,33.34,44.40,75.20,47,1010,4.70,0,Iraq,Baghdad Hotel,Baghdad,3.9
1,katsuura,35.13,140.30,71.78,51,1019,5.99,0,Japan,Katsuura Hilltop Hotel & Residence,"441-1 Sawakura, Katsuura",3.6
2,esna,25.29,32.55,71.60,43,1012,4.70,0,Egypt,Dahabeya Queen Farida Sailing Boat - Esna/Aswa...,"In Front of Luxor Temple, West Bank، Luxor",4.4
3,khawhai,23.37,93.12,74.70,40,1012,3.96,0,India,,,
4,jinchang,38.50,102.17,79.99,7,1010,6.35,0,China,龙首山庄,"16 Jinchuan W Rd, Jinchuan District, Jinchang",5
5,umm lajj,25.02,37.27,77.25,51,1010,1.59,0,Saudi Arabia,عروس الشاطئ للوحدات السكنية المفروشة,"King Abdullah Road, Al Dawayhi، Umluj",3.2
6,benguela,-12.58,13.41,76.82,71,1015,3.27,0,Angola,iu Hotel Benguela,"Av. Dr. António Agostinho Neto, Benguela",3.2
7,namibe,-15.20,12.15,75.94,62,1014,5.61,0,Angola,Hotel ibis Styles Iu Namibe,"Avenida Eduardo Mondlane, Namibe",3.6


In [13]:
 # Store latitude and longitude in locations
locations = df[["latitude", "longitude"]]

# Fill NaN values and convert to float
humid = df["humidity"]

In [14]:
city_info = []

for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Hotel Name</dt><dd>{row.hotels}</dd>
                <dt>City</dt><dd>{row.cities}</dd>
                <dt>Country</dt><dd>{row.country}</dd>
                </dl>
                """
    
    city_info.append(info_box)

In [16]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=True)

# Format
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
heat_layer.max_intensity = 100
heat_layer.point_radius = 5
heat_layer.gradient = ['silver', 'blue', 'red']
heat_layer.opacity = 0.3
gmaps.figure(layout=figure_layout)


# Add layer
fig.add_layer(heat_layer)

#add points
coordinates = ideal_cities[["latitude", "longitude"]]
marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…